In [13]:
import pandas as pd
import numpy as np
import re
import os
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [2]:
source = 'source/EAPEP - complete dataset.txt'
desc = 'source/Variable decription.txt'

In [4]:
data = pd.read_csv(source, delimiter='\t', encoding='iso-8859-1')

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
data0 = data.loc[:, :'MFPR']

In [10]:
txt = open(desc).read()

In [12]:
txt.split('\n')[3]

'sort_code                        Country code (sorting)'

In [18]:
re.split(' {2,}', txt.split('\n')[3])

['sort_code', 'Country code (sorting)']

In [19]:
conc = []
cname = []

for i in txt.split('\n')[3:]:
    sp = re.split(' {2,}', i)
    conc.append(sp[0])
    cname.append(sp[1])

In [21]:
conc.index('MFPR')

16

In [23]:
cdf = pd.DataFrame([], columns=['concept', 'concept_type', 'name'])

In [26]:
cdf['concept'] = conc[:17]
cdf['name'] = cname[:17]

In [28]:
cdf['concept_type'] = 'string'

cdf['concept_type'].iloc[6:] = 'measure'

cdf['concept_type'].iloc[4] = 'time'
cdf['concept_type'].iloc[5] = 'entity_domain'

cdf['concept'].iloc[1] = 'name'
cdf['name'].iloc[1] = 'Name'

cdf['concept'].iloc[3] = 'country'

In [30]:
cdf['concept'] = cdf['concept'].map(to_concept_id)

In [31]:
cdf

,concept,concept_type,name
0,sort_code,string,Country code (sorting)
1,name,string,Name
2,iso2,string,ISO country code (2 letters)
3,country,string,ISO country code (3 letters)
4,year,time,Year
5,age_group,entity_domain,Age group
6,mpop,measure,Population (M)
7,fpop,measure,Population (F)
8,mfpop,measure,Population (M+F)
9,mlf,measure,Labour Force (M)


In [32]:
cdf.to_csv('../ddf--concepts.csv', index=False)

In [39]:
country = data0[['iso3', 'iso2', 'country', 'sort_code']].copy()

In [40]:
country = country.drop_duplicates()

In [41]:
country.columns = ['country', 'iso2', 'name', 'sort_code']

In [42]:
country['country'] = country['country'].map(to_concept_id)

In [44]:
country.to_csv('../ddf--entities--country.csv', index=False)

In [51]:
age_group = data0[['age_group']].copy()

In [52]:
age_group['name'] = 'Aged ' + age_group['age_group']

In [53]:
age_group = age_group.drop_duplicates()

In [54]:
age_group['age_group'] = age_group['age_group'].map(to_concept_id)

In [56]:
age_group.to_csv('../ddf--entities--age_group.csv', index=False)

In [57]:
dps_col = [*['iso3', 'year', 'age_group'], *cdf['concept'].values[6:]]

In [59]:
data0.columns = list(map(to_concept_id, data0.columns))

In [70]:
dps = data0[dps_col].copy()

In [71]:
dps['age_group'] = dps['age_group'].map(to_concept_id)
dps['iso3'] = dps['iso3'].map(to_concept_id)

In [72]:
dps = dps.rename(columns={'iso3': 'country'})

In [73]:
dps = dps.set_index(['country', 'year', 'age_group'])

In [76]:
for i, df in dps.items():
    path = os.path.join('../', 'ddf--datapoints--{}--by--country--age_group--year.csv'.format(i))
    
    df = df.dropna().reset_index()
    
    df.to_csv(path, index=False)